# ML101:  A short course on machine learning

## Malcolm C. A. White and Nori Nakata

*Machine learning* (ML) refers to a diverse set of computer algorithms that are designed to "learn" how to make inferences from input data. ML is a component of Artifical Intelligence (AI), the study of reasoned decision making in machines.

In this short course, we will look at two main problems that ML algorithms attempt to solve: a) **classification** and b) **regression**. There are many other problems that ML algorithms attempt to solve, but we will limit ourselves to this pair of related problems for this short course. For our purposes, we will treat classification problems as those where we wish to infer the value of a categorical or discrete, numerical variable from a set of independent or explanatory variables. We will treat regression problems as those where we wish to infer the value of a continuous, numerical variable from a set of explanatory variables.

In service of solutions to these classes of problems, we will investigate various *supervised learning* methods, including *Artificial Neural Networks* (ANN), for solving basic classification and regression problems. As an example of a *Dimensionality Reduction* technique, we will also look at *Principal Component Analysis (PCA)* as a tool for building Image Recognition algorithms.

We begin by import various packages we will need.  Most notably, we will be using [`sckit-learn`](https://scikit-learn.org/stable/index.html) (`sklearn` for short) to build our ML models. Much of the code in this notebook is modeled after code in the `sklearn` documentation.

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import sklearn.cluster
import sklearn.datasets
import sklearn.decomposition
import sklearn.linear_model
import sklearn.metrics
import sklearn.mixture
import sklearn.neighbors
import sklearn.neural_network
import sklearn.svm
import sklearn.tree

import seaborn as sns

sns.set_theme()

We also need to define some functions to summarize and visualize our data and analysis results. The code cell below does this.

In [ ]:
def display_report(X, y, clf, title=None):
    print(
        sklearn.metrics.classification_report(
            y, 
            clf.predict(X), 
            labels=clf.classes_)
    )
    fig, ax = plt.subplots(figsize=(8, 6))
    sns.heatmap(
        sklearn.metrics.confusion_matrix(y, clf.predict(X), normalize="true"), 
        annot=True,
        ax=ax,
        xticklabels=clf.classes_,
        yticklabels=clf.classes_
    )
    ax.set_xlabel("Predicted label")
    ax.set_ylabel("True label")
    if title is not None:
        fig.suptitle(title)
        
    plt.tight_layout()

    
def plot_decision_function(X, y, clf, title=None, palette="icefire"):
    _palette = sns.color_palette(palette)
    for i in range(1, len(_palette)-1):
        _palette.remove(_palette[1])
        
    fig, ax = plt.subplots()
    sns.scatterplot(
        data=X.merge(y, left_index=True, right_index=True), 
        x="feature 1",
        y="feature 2",
        hue="target",
        hue_order=sorted(y.unique()),
        palette=_palette,
        ax=ax
    )
    xx, yy = np.meshgrid(
        np.linspace(*ax.get_xlim()),
        np.linspace(*ax.get_ylim())
    )
    xy = np.stack([xx.flatten(), yy.flatten()]).T
    z = clf.decision_function(xy)

    ax.pcolormesh(
        xx,
        yy,
        z.reshape(xx.shape),
        zorder=0,
        cmap=sns.color_palette(palette, as_cmap=True),
        shading="gouraud"
    )
    

def plot_gallery(images, h, w, titles=None, nrows=3, ncols=4):
    figsize = (1.5*ncols, 1.5*nrows*h/w)
    fig, axes = plt.subplots(figsize=figsize, nrows=nrows, ncols=ncols)
    for i, ax in enumerate(axes.flatten()):
        ax.imshow(images[i].reshape((h, w)), cmap=plt.cm.gray)
        if titles is not None:
            ax.set_title(titles[i])
        ax.axis("off")
    plt.tight_layout()


def plot_joint(X, y, palette="icefire", nclasses=2):
    if nclasses == 2:
        _palette = sns.color_palette(palette)
        for i in range(1, len(_palette)-1):
            _palette.remove(_palette[1])
    else:
        _palette = palette
        
    g = sns.JointGrid(
        data=X.merge(y, left_index=True, right_index=True), 
        x="feature 1",
        y="feature 2",
        hue="target",
        hue_order=sorted(y.unique()),
        palette=_palette,
        space=0
    )
    g.plot_joint(
        sns.scatterplot,
    )
    g.plot_marginals(
        sns.histplot, 
        kde=True,
        alpha=1/2,
        bins=32
    )
    

def plot_pairs(X, y, model=None, title=None):
    fig, axes = plt.subplots(
        figsize=(12, 6), 
        nrows=2, 
        ncols=4, 
        sharey=True
    )
    i = 0
    for column, ax in zip(sorted(X.columns), axes.flatten()):
        sns.scatterplot(
            x=X[column],
            y=y,
            s=16,
            ax=ax
        )
        if model is not None:
            sns.scatterplot(
                x=X[column],
                y=model.predict(X),
                s=16,
                ax=ax
            )
        xmin, xmax = X[column].quantile([0.01, 0.99])
        dx = 0.02 * (xmax - xmin)
        ax.set_xlim(xmin-dx, xmax+dx)
            
    if title is not None:
        if model is not None:
            sqerr = sklearn.metrics.mean_squared_error(
                y_test, 
                model.predict(X_test)
            )
            fig.text(0.98, 0.98, f"$\epsilon={sqerr:.3f}$", va="top", ha="right")
        fig.suptitle(title)
    if model is not None:
        handles = axes[0, 0].get_children()[:2]
        labels = ["True", "Predicted"]
        fig.legend(handles, labels, loc=2)
        
    plt.tight_layout()
    
    return (fig, axes)

# **1. Classification (Supervised)**

In this section on Classification, we will compare the performance of three classification algorithms: a) k-Nearest Neighbours, b) Support Vector Machines, and c) Random Forests.

## **1.1 Data**

To build and test our different classification models, we will work with a 2-D, synthetic data set with two classes. In other words, we want to perform binary classification using just two features. The methods we will look at generalize to data from multiple classes with more than two features, although the computational costs grow at different rates for the various algorithms. It is simplest to visualize data with only two classes in two dimensions, so we will stick to such data. The code below will create a synthetic data set and split it into *training* and *test* data subsets, each with half of the data.

In [ ]:
X, y = sklearn.datasets.make_moons(n_samples=1024, shuffle=True, noise=1/8, random_state=None)

X = pd.DataFrame(X, columns=[f"feature {i+1}" for i in range(X.shape[1])])
y = pd.Series(y+1, name="target")
y = "label " + y.astype(str)
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.5)

As a first step, we can plot and look at the entire data set.

In [ ]:
plot_joint(X, y)

We can also plot just the data in our training data set.

In [ ]:
plot_joint(X_train, y_train)

Or just the data in our test data set.

In [ ]:
plot_joint(X_test, y_test)

## **1.1 *k*-Nearest Neighbours**

For our first model, we will use the [*k*-Nearest Neighbours](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier) algorithm. It is a relatively simple matter to build and train our model; however, we are naively using the default hyper-parameters here, which may not be appropriate for our data set.

In [ ]:
clf = sklearn.neighbors.KNeighborsClassifier()
clf.fit(X_train, y_train);

Nevertheless, let's see how our model performs. First, we can plot the inferred (predicted) class for each of our test cases.

In [ ]:
plot_joint(X_test.reset_index(drop=True), pd.Series(clf.predict(X_test), name="target"))

And we can score our model based on *precision*, *recall*, and *f1-score*. The code below also plots a [*confusion matrix*](https://en.wikipedia.org/wiki/Confusion_matrix).

In [ ]:
plt.close("all")
display_report(X_test, y_test, clf, title="k-Nearest Neighbours Classifier")

It is nearly perfect. The *k*-Nearest Neighbours algorithm seems well-suited for this particular data set.

## **1.2 Support-Vector Machines (SVM)**
An alternative approach to solving the classification problem is using *Support Vector Machines* (SVM). Below, we fit an SVM with a simple linear kernel to the training data.

In [ ]:
clf = sklearn.svm.SVC(kernel="linear")
clf.fit(X_train, y_train);

The code below plots the *decision function* for the SVM model.

In [ ]:
plot_decision_function(X_test, y_test, clf, title="SVM with linear kernel")

You can see that the decision function separates the classes poorly. We can see this qualitatively in the plot below.

In [ ]:
plot_joint(X_test.reset_index(drop=True), pd.Series(clf.predict(X_test), name="target"))

Generating our performance report, we see this reflected quantitatively.

In [ ]:
display_report(X_test, y_test, clf, title="SVM with linear kernel")

We can repeat the analysis using a higher-order polynomial basis function.

In [ ]:
clf = sklearn.svm.SVC(kernel="poly")
clf.fit(X_train, y_train)
plot_decision_function(X_test, y_test, clf, title="SVM with linear kernel")
display_report(X_test, y_test, clf, title="SVM with polynomial kernel")

The performance is better, but still not as good as the k-Nearest Neighbours model. We can also try using a Radial Basis Function kernel.

In [ ]:
clf = sklearn.svm.SVC(kernel="rbf")
clf.fit(X_train, y_train)
plot_decision_function(X_test, y_test, clf, title="SVM with linear kernel")
display_report(X_test, y_test, clf, title="SVM with RBF kernel")
plot_joint(X_test.reset_index(drop=True), pd.Series(clf.predict(X_test), name="target"))

## **1.3 Random Forests**

As a final example, we compare our previous results with a Random Forest Classifier.

In [ ]:
clf = sklearn.tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)

display_report(X_test, y_test, clf, title="Random Forest Classifier")

The Random Forest model has the best performance of all the models.

## **1.4 Exercise**

Try replacing the first line of Section 1.1 with one of the lines below, and repeat the analysis using a different data set. Is the relative performance of the models the same for different data sets?

In [ ]:
X, y = sklearn.datasets.make_circles(n_samples=1024, noise=1/32)
X, y = sklearn.datasets.make_blobs(n_samples=1024, centers=2, cluster_std=2)
X, y = sklearn.datasets.make_gaussian_quantiles(n_samples=1024, n_classes=2)

# **2. Classification (Unsupervised)**

In unsupervised learning methods, class labels are unknown _a priori_. Unsupervised learning algorithms must independently discover patterns in the training data set. We will look at two unsupervised classification methods: a) K-Means Clustering, and b) Gaussian Mixture Models.

# **2.1 Data**

We will begin with a synthetic data set like the one we started with in Section 1. In this synthetic example, we know the true labels of our data, but this information cannot be used by the unsupervised methods.

After completing this section, try using one of the alternative data sets from Section 1. When you do so, note that you can change the `centers` and `n_classes` keyword arguments of the `make_blobs()` and `make_gaussian_quantiles()` functions, respectively.

In [ ]:
X, y = sklearn.datasets.make_moons(n_samples=1024, shuffle=True, noise=1/8, random_state=None)

X = pd.DataFrame(X, columns=[f"feature {i+1}" for i in range(X.shape[1])])
y = pd.Series(y+1, name="target")
y = "label " + y.astype(str)
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.5)

plot_joint(X, y)

Generally speaking, unsupervised learning methods need a way to determine how many unique classes are represented within the data. The simplest way to solve this problem is to assume a certain number of classes. This is the approach we will take. After running the examples in this section, try changing the value of `n_classes` below to see how it affects your analysis.

In [ ]:
n_classes = 5

## **2.1 K-Means Clustering**

In [ ]:
clf = sklearn.cluster.KMeans(n_clusters=n_classes)
clf = clf.fit(X_train, y_train)
plot_joint(
    X_test.reset_index(drop=True), 
    pd.Series(clf.predict(X_test), name="target"),
    nclasses=n_classes,
    palette="colorblind"
)

## **2.2 Gaussian Mixture Models**

In [ ]:
clf = sklearn.mixture.GaussianMixture(n_components=n_classes)
clf = clf.fit(X, y)
plot_joint(
    X_test.reset_index(drop=True), 
    pd.Series(clf.predict(X_test), name="target"),
    nclasses=n_classes,
    palette="colorblind"
)

Evaluating the performance of these methods is made challenging by the lack of labeled test data.

# **3. Regression**

In this section on Regression, we will compare the performance of a Linear Regression model with an Artificial Neural Network. 

## **3.1 Data**
We will use a data set containing information taken from a census on housing attributes in California, USA (https://scikit-learn.org/stable/datasets/real_world.html#california-housing-dataset). The following is an excerpt on the data set from the `scikit-learn` documentation.

```
This dataset was obtained from the StatLib repository. https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts, expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived from the 1990 U.S. census, using one row per census block group. A block group is the smallest geographical unit for which the U.S. Census Bureau publishes sample data (a block group typically has a population of 600 to 3,000 people).

An household is a group of people residing within a home. Since the average number of rooms and bedrooms in this dataset are provided per household, these columns may take surpinsingly large values for block groups with few households and many empty houses, such as vacation resorts.

It can be downloaded/loaded using the sklearn.datasets.fetch_california_housing function.

Pace, R. Kelley and Ronald Barry, Sparse Spatial Autoregressions, Statistics and Probability Letters, 33 (1997) 291-297
```

The code below will load the data, split it into *training* and *test* data sets (80% and 20% of the data, respsectively).

In [ ]:
data = sklearn.datasets.fetch_california_housing(as_frame=True)
X = data["data"]
y = data["target"]
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.2)

As before, we will plot the entire data set, the training data, and the test data.

In [ ]:
fig, axes = plot_pairs(X, y, title="All data")
fig, axes = plot_pairs(X_train, y_train, title="Training data")
fig, axes = plot_pairs(X_test, y_test, title="Test data")

## **3.1 Linear Regression**

Linear Regression models are likely the most familiar, and the simplest type of Linear Regression model uses the $\ell_2$-minimizing hyperplane to predict the target variable from the explanatory variables.

The code below fits a simple least-squares Linear Regression model to the data and plots a comparison of the True versus Predicted values for the test data set. The mean square error, $\varepsilon$, is reported in the top right corner.

Note that the least-square Linear Regression model is deterministic, and running the code below multiple times will produce identical output.

In [ ]:
model = sklearn.linear_model.LinearRegression()
model.fit(X_train, y_train);

fig, axes = plot_pairs(X_test, y_test, model=model, title=f"Linear Regression Model")

## **3.2 Artifical Neural Networks**

Artificial Neural Networks (ANNs) are popular today for their ability to approximate highly non-linear relationships between variables. To achieve this, they use a sequence of connected layers of nodes that mimic the structure and function of nerouns and synapses in the human brain.

The code below will create an ANN with a single hidden layer of nodes with [ReLu](https://en.wikipedia.org/wiki/Rectifier_(neural_networks)) activation functions.

Note that the model parameters for ANNs are stochastically determined, so running the code below multiple times will produce different output each time.

In [ ]:
hidden_layer_sizes = (100,)
activation         = "relu"

model = sklearn.neural_network.MLPRegressor(
    activation=activation, 
    hidden_layer_sizes=hidden_layer_sizes
)
model.fit(X_train, y_train)

fig, axes = plot_pairs(X_test, y_test, model=model, title="Multi-Layer Perceptron Model\n(ReLu activation)")

We see that the much simpler Linear Regression model actually outperforms the relatively complex ANN. We can, however, try a different activation function. The code below creates an ANN with hyperbolic tangent activation functions.

In [ ]:
hidden_layer_sizes = (100,)
activation         = "tanh"

model = sklearn.neural_network.MLPRegressor(
    activation=activation,
    hidden_layer_sizes=hidden_layer_sizes
)
model.fit(X_train, y_train)

fig, axes = plot_pairs(X_test, y_test, model=model, title="Multi-Layer Perceptron Model\n(tanh activation)")

The tanh activation function outperforms the ReLu activation function in this application.

We can also increase the number of hidden layers and/or the size of each hidden layer. The code below creates an ANN with two hidden layers (one with 128 nodes and one with 64 nodes) using hyperbolic tanget activation functions.

In [ ]:
hidden_layer_sizes = (128, 64)
activation         = "tanh"

model = sklearn.neural_network.MLPRegressor(
    activation=activation,
    hidden_layer_sizes=hidden_layer_sizes
)
model.fit(X_train, y_train)

fig, axes = plot_pairs(X_test, y_test, model=model, title="Multi-Layer Perceptron Model\n(tanh activation)")

Does this more complex ANN outperform the smaller ANN with just a single layer of 100 nodes?


## **3.3 Exercise**
Try changing the number and size of hidden layers and the activation function in the code below to see if you can build a better ANN.

In [ ]:
hidden_layer_sizes = (128, 64, 128)
activation         = "tanh" # Try "tanh", "relu", and "logistic"

model = sklearn.neural_network.MLPRegressor(
    activation=activation,
    hidden_layer_sizes=hidden_layer_sizes
)
model.fit(X_train, y_train)

fig, axes = plot_pairs(
    X_test, 
    y_test, 
    model=model, 
    title=f"Multi-Layer Perceptron Model\n"
    f"{activation} activation\n"
    f"Hidden layer sizes: {hidden_layer_sizes}"
)

# **4. Image Recognition**

Finally, we will build a facial recognition model.

## **4.1 Data**

For this, we will use the ["Labeled Faces in the Wild"](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_lfw_people.html) data set, which contains images of faces of several well-known world leaders.

In [ ]:
faces = sklearn.datasets.fetch_lfw_people(min_faces_per_person=70, resize=0.4)
X = faces["data"]
y = faces["target_names"][faces["target"]]
n_samples, h, w = faces["images"].shape

In [ ]:
plot_gallery(X, h, w, titles=y, nrows=4, ncols=4)

Next we split the data in training and test data sets. The test data set comprises 25% of the data.

In [ ]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.25)

## **4.2 Dimensionality Reduction: Principal Component Analysis (PCA)**

First, we need to extract features to represent each image. This can be achieved using Principal Component Analysis to reduce the dimensionality of the input data. We will reduce each image to a set of 128 coefficients, each of which is associated with an *eigenface*. Each of the original images can be approximately reconstructed by multiplying the eigenfaces by their respective coefficients and summing. The eigenfaces represent an orthogonal basis that spans (within some error) the space of faces in the input.

The code below will compute and plot (a portion of) the set of eigenfaces.

In [ ]:
n_components=128

pca = sklearn.decomposition.PCA(
    n_components=n_components, 
    svd_solver="randomized",
    whiten=True
)
pca.fit(X_train)

eigenfaces = pca.components_.reshape((n_components, h, w))

plot_gallery(eigenfaces, h, w, nrows=7, ncols=10)

Having computed the spanning set of eigenfaces, we can decompose the training and test data as linear combinations of them.

In [ ]:
W_train = pca.transform(X_train)
W_test  = pca.transform(X_test)

Each image is now represented by a vector of 128 eigenface coefficients.

Using an SVM with a RBF kernel as before, we can build our classification model.

In [ ]:
clf = sklearn.svm.SVC(kernel='rbf', class_weight='balanced')
clf.fit(W_train, y_train);

We can visually inspect our inferences.

In [ ]:
y_pred = clf.predict(W_test)
plot_gallery(X_test, h, w, titles=y_pred, nrows=4, ncols=4)

And finally, we can display a performance report for our model.

In [ ]:
display_report(W_test, y_test, clf)

How did the model perform?